In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline  
pd.set_option('display.max_columns', None)
from sklearn.linear_model import SGDClassifier
import sklearn.cross_validation
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics 

In [2]:
TITLE = ['Position', 'HistCTR', 'IsClick', 'IsUserLoggedOn', 'Price']
SOURCE = "../data/train_sampled.csv"

In [3]:
df = pd.read_csv(SOURCE , sep=',' , usecols=TITLE) 
label_frame = df['IsClick']
df = df.drop(['IsClick'],axis=1)

In [16]:
df.describe()

,Position,HistCTR,IsUserLoggedOn,Price
count,3438867.000000,3438867.000000,3438867.000000,3.437818e+06
mean,3.385109,0.012274,0.307141,1.889284e+04
std,2.936309,0.016786,0.461308,1.484530e+05
min,1.000000,0.000010,0.000000,0.000000e+00
25%,1.000000,0.002967,0.000000,1.390000e+03
50%,1.000000,0.006735,0.000000,4.429000e+03
75%,7.000000,0.014599,1.000000,1.300000e+04
max,7.000000,1.000000,1.000000,1.300000e+08


In [21]:
df[df.Price.isnull()]
prices_median = df.Price.median()

In [75]:
df.Price.fillna(0, inplace=True)

In [27]:
df.Price.head()

0     1000
1     3000
2     5470
3    10419
4      878
Name: Price, dtype: float64

In [61]:
scaler = MinMaxScaler()
df.Price = scaler.fit_transform(df.Price.reshape(-1, 1))

In [62]:
df.Position = scaler.fit_transform(df.Position.reshape(-1, 1))

In [76]:
clf = SGDClassifier(loss="log")
scores = cross_val_score(clf, df, label_frame, cv=10, scoring='log_loss')
scores.mean()

-0.62318613924403632

In [34]:
clf.fit(df, label_frame)
clf.predict_proba(df)
clf.score(df, label_frame )

0.66758295682851354

In [39]:
sklearn.metrics.log_loss(label_frame, clf.predict_proba(df))

0.62316994215069021

In [73]:
inc_clf = SGDClassifier(loss="log")
scaler = MinMaxScaler()
for chunk in pd.read_csv(SOURCE, sep=',', usecols=TITLE , chunksize=100000):
    Y = chunk['IsClick']
    X = chunk.drop(['IsClick'],axis=1)
    X.Price.fillna(X.Price.median(), inplace=True)
    X.Price = scaler.fit_transform(X.Price.reshape(-1, 1))
    #X.Position = scaler.fit_transform(X.Position.reshape(-1, 1))
    inc_clf.partial_fit(X, Y ,  classes=[0,1])
    

In [74]:
sklearn.metrics.log_loss(label_frame, inc_clf.predict_proba(df))

1.0725473847716036

In [4]:
df = pd.read_csv(SOURCE , sep=',' ) 
label_frame = df['IsClick']
df = df.drop(['IsClick'],axis=1)

In [5]:
df.head()

,SearchID,AdID,Position,HistCTR,SearchDate,IPID,UserID,IsUserLoggedOn,SearchQuery,SearchLocationID,SearchCategoryID,SearchParams,SearchLocationLevel,SearchRegionID,SearchCityID,SearchCategoryLevel,SearchParentCategoryID,SearchSubcategoryID,UserAgentID,UserAgentOSID,UserDeviceID,UserAgentFamilyID,AdCategoryID,Params,Price,Title,AdCategoryLevel,AdParentCategoryID,AdSubcategoryID
0,121,1220223,7,0.005422,2015-05-09 22:21:40.0,490755,3817669,0,NaN,2685,44,NaN,3,9,1582,3,12,22,1582,44,2014,62,44,{223:'Бинокли и телескопы'},1000,"Лупа,монокуляр на очковой оправе,Отправка по Р...",3,12,22
1,181,13840581,7,0.013889,2015-04-30 10:24:49.0,1507803,2391097,0,NaN,230,22,"{83:'Платья и юбки', 175:'Женская одежда', 90:...",3,2,2951,3,3,47,63091,20,2014,25,22,"{83:'Платья и юбки', 175:'Женская одежда'}",3000,Брендовые вечерние платья,3,3,47
2,198,14515103,1,0.033885,2015-05-07 10:46:21.0,951832,372668,1,chevrolet cruze,3960,34,"{598:'Для автомобилей', 5:'Запчасти'}",2,18,623,3,6,7,54509,44,2014,62,34,"{5:'Запчасти', 598:'Для автомобилей'}",5470,Запчасти б/у и новые для chevrolet cruze (09-),3,6,7
3,404,11801992,1,0.007371,2015-04-30 16:48:21.0,130853,2621226,0,NaN,2473,49,NaN,3,14,698,2,8,45,18318,20,2014,25,46,NaN,10419,Инкубатор Блиц 48 цифровой,3,8,40
4,438,29135166,1,0.004846,2015-05-01 15:48:08.0,599651,2675985,1,NaN,2060,35,NaN,2,44,1890,3,9,50,10275,20,2014,25,35,NaN,878,Аэросад Луковое счастье,3,9,50
